In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer

from fuzzywuzzy import process
from fuzzywuzzy import fuzz

In [146]:
lic = pd.read_csv('data/raw/Copia de AGOSTO COMPRAS MUNICIPALES 2016-2019-SOCIAL TIC.xlsx - Compras.csv').dropna(subset=['PROVEEDOR'])
soc = pd.read_csv('data/processed/socios_padron_proveedores_long.csv')

In [147]:
pd.read_csv('data/processed/compras_complete_cases_graph.csv').sort_values('PROVEEDOR').head()

,id,PROVEEDOR,UNIDAD.ADMINISTRATIVA.CONVOCANTE,.MONTO.TOTAL.O.MÁXIMO
869,870,26 DE MEXICO,OFICIALÍA MAYOR,NaN
152,153,26 DE MEXICO,OFICIALÍA MAYOR,790000.0
138,139,26 DE MEXICO,OFICIALÍA MAYOR,720000.0
799,800,399 PROJEY DEVELOPMENT,OFICIALÍA MAYOR,3572800.0
333,334,"AALIANZA, INGENIERIA Y CONSTRUCCION",OFICIALÍA MAYOR,5051800.0


In [148]:
lic = lic.assign(prov=list(
    lic
    .PROVEEDOR
    .str.lower()
    .str.replace('s\.a\..*', '')
    .str.replace('sa\ de\ .*', '')
    .str.findall(pat='([a-záéíóúñ0-9]+)')
    .apply(lambda x: ' '.join(x))
))

In [149]:
soc = soc.assign(
    emps=
        soc['Nombre de la empresa']
        .str.lower()
        .str.replace('s\.a\..*', '')
        .str.replace('sa\ de\ .*', '')
        .str.replace('á', 'a')
        .str.replace('é', 'e')
        .str.replace('í', 'i')
        .str.replace('ó', 'o')
        .str.replace('ú', 'u')
        .str.findall(pat='([a-ñ0-9]+)')
        .apply(lambda x: ' '.join(x)))

In [150]:
tfidf = TfidfVectorizer()
tfidf.fit(pd.Series(lic.prov.tolist() + soc.emps.tolist()))

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [151]:
def limit_tfidf(string, model, lim):
    return pd.DataFrame.from_dict(
        dict(model.transform([string]).todok()),
             orient='index').reset_index().assign(
            row=lambda x: x['index'].str[0],
            col=lambda x: x['index'].str[1]).assign(
            palabra=lambda x: x.col.apply(lambda y: list(model.get_feature_names())[y]),
        ).rename(columns={0: 'tfidf'}).query('tfidf > '+str(lim)).palabra.str.cat(sep=' ')

In [152]:
lic = lic.assign(prov_tfidf=lambda x: x.prov.apply(lambda y: limit_tfidf(y, tfidf, 0.4)))
soc = soc.assign(emp_tfidf=lambda x: x.emps.apply(lambda y: limit_tfidf(y, tfidf, 0.4)))

In [153]:
proveedores_aux = lic[~lic.dropna(subset=['PROVEEDOR'])['PROVEEDOR'].str.startswith('C. ')][['PROVEEDOR', 'prov_tfidf']].drop_duplicates()

proveedores = list(proveedores_aux.prov_tfidf)

In [154]:
emps_aux = soc[['Nombre de la empresa', 'emp_tfidf']].drop_duplicates()

emps = list(emps_aux.emp_tfidf)

In [155]:
emp = []
score = []

for p in proveedores:
    aaa = pd.DataFrame({
        'e': emps
    }).assign(score=lambda x: x.e.apply(lambda y: fuzz.token_sort_ratio(p, y))).sort_values('score', ascending=False)
    emp.append(aaa.e.iloc[0])
    score.append(aaa.score.iloc[0])

In [156]:
cruce_prov_emp = pd.DataFrame({
    'emp_tfidf': emp,
    'prov': proveedores,
    'PROVEEDOR': proveedores_aux.PROVEEDOR,
    'score': score
}).merge(emps_aux).sort_values('score').query('score > 81')[['PROVEEDOR', 'Nombre de la empresa']]

In [227]:
(
    lic[['TITULAR DE LA DEPENDENCIA SOLICITANTE', 'PROVEEDOR']]
#     [~lic.PROVEEDOR.str.contains('C. ')]
    .reset_index().rename(columns={'index': 'id_lic'})
    .merge(cruce_prov_emp, how='left')
    .merge(soc[['Nombre de la empresa', 'Propietarios']], how='left')
#     .assign(Propietarios=lambda x: 
#         x.Propietarios.mask(x.Propietarios.isna() & x.PROVEEDOR.str.startswith('C.'), x.PROVEEDOR.str.lower().str[2:]))
    .dropna(subset=['TITULAR DE LA DEPENDENCIA SOLICITANTE', 'Propietarios'])
    .assign(
        apell_titular=lambda x: x['TITULAR DE LA DEPENDENCIA SOLICITANTE'].str.lower().str.split().str[-2:].str.join(' '),
        apell_prop=lambda x: x['Propietarios'].str.lower().str.split().str[-2:].str.join(' '))
    .assign(score_fuzzy=lambda x: x.apply(lambda y: 
        fuzz.token_set_ratio(
            y.apell_titular,
            y.apell_prop),
        axis=1))
    .assign(score_directo=lambda x: x.apply(lambda y:
        max(y.apell_titular.split()[0] == y.apell_prop.split()[0],
           y.apell_titular.split()[0] == y.apell_prop.split()[1],
           y.apell_titular.split()[1] == y.apell_prop.split()[0],
           y.apell_titular.split()[1] == y.apell_prop.split()[1]), axis=1))
    .sort_values('score_fuzzy', ascending=False)
)

,id_lic,TITULAR DE LA DEPENDENCIA SOLICITANTE,PROVEEDOR,Nombre de la empresa,Propietarios,apell_titular,apell_prop,score_fuzzy,score_directo
1390,861,DANIEL PANDO MORALES,"PROYECTOS ELECTRICOS Y SERVICIOS, S.A. DE C.V.",Proyectos Eléctricos y Servicios,Guadalupe Hernández Morales,pando morales,hernández morales,70,True
1410,874,DANIEL PANDO MORALES,"PROYECTOS ELECTRICOS Y SERVICIOS, S.A. DE C.V.",Proyectos Eléctricos y Servicios,Guadalupe Hernández Morales,pando morales,hernández morales,70,True
1354,840,FRANCISCO JAVIER ARROYOS SALGADO,"PROYECTOS Y CONSTRUCCIONES VALAR, S.A. DE C.V.",Proyectos y Construcciones Valar,Luis Carlos Arroyo Estrada.,arroyos salgado,arroyo estrada.,69,False
1447,900,FRANCISCO JAVIER ARROYOS SALGADO,"PROYECTOS Y CONSTRUCCIONES VALAR, S.A. DE C.V.",Proyectos y Construcciones Valar,Evangelina Arroyo estrada,arroyos salgado,arroyo estrada,69,False
1467,911,FRANCISCO JAVIER ARROYOS SALGADO,"CONSTRUCTORA ELECTRICA FER, S.A. DE C.V.",Constructora Eléctrica Fer,Alicia Arroyo Rosales,arroyos salgado,arroyo rosales,69,False
1450,900,FRANCISCO JAVIER ARROYOS SALGADO,"PROYECTOS Y CONSTRUCCIONES VALAR, S.A. DE C.V.",Proyectos y Construcciones Valar,Luis Carlos Arroyo Estrada.,arroyos salgado,arroyo estrada.,69,False
1351,840,FRANCISCO JAVIER ARROYOS SALGADO,"PROYECTOS Y CONSTRUCCIONES VALAR, S.A. DE C.V.",Proyectos y Construcciones Valar,Evangelina Arroyo estrada,arroyos salgado,arroyo estrada,69,False
34,25,MÓNICA LETICIA LUÉVANO GARCÍA,"TELEVISIÓN DE LA FRONTERA, S.A.","Televisión de la frontera, Sociedad Anónima",Ricardo Garza Álvarez,luévano garcía,garza álvarez,67,False
1462,908,FRANCISCO JAVIER ARROYOS SALGADO,GRUPO ASESOR EN INNOVACIÓN TECNOLÓGICA Y EDUCA...,Grupo Asesor en Innovación Tecnológica y Educa...,Liliana Arroyo Ríos,arroyos salgado,arroyo ríos,64,False
1486,923,FRANCISCO JAVIER ARROYOS SALGADO,"CONSTRUCTORA DE OBRA Y URBANIZACIÓN, S.A. DE C.V.",Constructora de Obra y Urbanización,Efraín del Pardo Salais,arroyos salgado,pardo salais,59,False
